The main gotal of this notebook is to prototype the text generation engine to explain why the retrieved job vacancy is a good fit for the user's resume.

To this end, it will be used LangChain and local LLMs from the Hugging Face Hub.

References:

- [Hugging Face Local Pipelines](https://python.langchain.com/docs/integrations/llms/huggingface_pipelines/)

In [1]:
# pip install --upgrade --quiet transformers torch --quiet

In [2]:
import os
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_core.prompts import PromptTemplate
import torch
import transformers
transformers.logging.set_verbosity_error()

/Users/luisrodrigues/miniconda3/envs/resume-worth/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Change the current working directory to the pachage root
# That's step is due to the way settings.py is defined
ROOT_DIR = os.path.join(*os.path.split(os.getcwd())[:-1])
os.chdir(ROOT_DIR)
os.getcwd()

'/Users/luisrodrigues/Documents/Projects/PERSONAL/resume-worth'

In [47]:
# Tested local models
# gpt2, gpt2-large, gpt2-xl 
# llmware/bling-1.4b-0.1, llmware/bling-sheared-llama-1.3b-0.1, llmware/bling-phi-2-v0, llmware/bling-red-pajamas-3b-0.1
# stabilityai/stablelm-3b-4e1t
# M4-ai/tau-0.5B-instruct, M4-ai/tau-1.8B, M4-ai/Hercules-Mini-1.8B
model_id = "M4-ai/tau-1.8B"  

# See instructions for parameters: https://www.ibm.com/docs/en/watsonx-as-a-service?topic=lab-model-parameters-prompting
top_p=0.7
top_k=30 
temperature=0.3
max_new_tokens=384

prompt_dir = os.path.join("data", "04_prompts")
promp_file = "prompt_template_for_explaning_why_is_a_good_fit.json"

In [31]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False, do_sample=True, top_p=top_p, top_k=top_k, 
                              temperature=temperature, max_new_tokens=max_new_tokens, num_beams=1, repetition_penalty=1.1, num_return_sequences=1)
hf = HuggingFacePipeline(pipeline=pipe)

In [42]:
if "llmware" in model_id:
    # See instructions for the prompt: https://huggingface.co/llmware/bling-sheared-llama-1.3b-0.1

    my_prompt = """JOB: {job} \n RESUME: {resume} \n Why is RESUME suitable for the JOB?"""

    template = "\<human>\: " + my_prompt + "\n" + "\<bot>\:"
elif "M4-ai" in model_id:
    # See instructions for the prompt: https://huggingface.co/spaces/Locutusque/Locutusque-Models/blob/main/app.py

    my_prompt = """Explain why the following RESUME is a good match for the presented JOB VACANCY.
    Keep your answer grounded in the facts of the RESUME and JOB VACANCY.
    Write a maximum of three points in clear and concise language.

    RESUME: 
    {resume}
    
    JOB VACANCY: 
    {job}"""

    template = f"<|im_start|>user\n{my_prompt}<|im_end|>\n<|im_start|>assistant\n"
else:
    template = """
    Explain why the following RESUME is a good match for the JOB below.
    Keep your answer ground in the facts of the RESUME and JOB.

    RESUME:
    {resume}

    JOB: 
    {job}
    """

prompt = PromptTemplate.from_template(template)

prompt

PromptTemplate(input_variables=['job', 'resume'], template='<|im_start|>user\nExplain why the following RESUME is a good match for the presented JOB VACANCY.\n    Keep your answer grounded in the facts of the RESUME and JOB VACANCY.\n    Write a maximum of three points in clear and concise language.\n\n    RESUME: \n    {resume}\n    \n    JOB VACANCY: \n    {job}<|im_end|>\n<|im_start|>assistant\n')

In [48]:
# my_prompt = """Explain why the following RESUME is a good match for the presented JOB VACANCY.
# Keep your answer grounded in the facts of the RESUME and JOB VACANCY.
# Write a maximum of three points in clear and concise language.
# 
# RESUME: 
# {resume}
# 
# JOB VACANCY: 
# {job}"""
# 
# template = f"<|im_start|>user\n{my_prompt}<|im_end|>\n<|im_start|>assistant\n"
# 
# prompt = PromptTemplate.from_template(template)
# 
# promp_path = os.path.join(prompt_dir, promp_file)
# prompt.save(promp_path)
# 
# from langchain.prompts import load_prompt
# 
# prompt = load_prompt(promp_path)

In [57]:
chain = prompt | hf

In [55]:
resume =  """Luis Antonio Rodrigues is an accomplished data scientist and machine learning engineer with over eight years of experience in developing innovative machine learning products and services. He holds a BSc in Mathematics, an MSc, and a PhD in Mechanical Engineering from the University of Campinas, one of the most renowned universities in Latin America. Luis's expertise spans across various domains including Natural Language Processing (NLP), Recommender Systems, Marketing and CRM, and Time-Series Forecasting, with significant contributions across Banking, Consumer Packaged Goods, Retail, and Telecommunications industries.
Currently serving as a Principal Data Scientist at DEUS, an AI firm dedicated to human-centered solutions, Luis plays a crucial role in the development of a cutting-edge Retrieval-Augmented Generation (RAG) solution. His responsibilities include improving the knowledge-to-text module, optimizing information retrieval for efficiency and precision, and enhancing text generation for real-time accuracy,  showcasing his skills in RAG, IR, LLM, NLP, and several tools and platforms. Additionally, he has contributed as a Data Architect in designing a medallion architecture for a Databricks lakehouse on AWS.
Previously, Luis held the position of Principal Data Consultant at Aubay Portugal, where he led an NLP project for Banco de Portugal, focusing on AI services such as summarization, information extraction, complaint text classification, and financial sentiment analysis. At CI&T, as Lead Data Scientist, he was instrumental in developing a recommender system for Nestlé, resulting in a 6% sales increase. During his time at Propz, he developed a recommender system for Carrefour, which boosted revenue by 3%.
His earlier roles include a researcher at I.Systems, focusing on water distribution systems, and at the University of Campinas, where his work centered on system and control theory. Luis's proficiency is further demonstrated by his certifications in MLOps with Azure Machine Learning, TensorFlow 2.0, and Python for Time Series Data Analysis. Luis combines his deep technical knowledge with strong communication skills to lead teams and projects towards achieving significant business impacts."""

job = """Design, develop, and deploy machine learning models and algorithms for complex and unique datasets, using various techniques such as mathematical modeling, scikit-learn, NLP, CNN, RNN, DL, RL, Transformers, GAN, LLM, RAG
Collaborate with cross-functional teams to extract insights, identify business opportunities and provide data-driven recommendations
Stay up-to-date with the latest machine learning and AI techniques and tools
Communicate complex technical concepts to non-technical stakeholders in an easy-to-understand manner
Bachelor's degree or higher in Computer Science, Mathematics, Statistics, Actuarial Science, Informatics, Information Science or related fields
Strong analytical skills and attention to detail
Participation in Kaggle, Mathematics Olympiad or similar competitions is a plus
Excellent programming skills in Python, R, Java, or C++\nFamiliar with ML frameworks such as Tensorflow, Keras, PyTorch, MLFlow, AutoML, TensorRT, CUDA
Excellent communication and collaboration skills\nExperience with designing, training, and deploying machine learning models
Customer centric and committed to deliver the best AI results to customers"""

In [63]:
answer = chain.invoke({"resume": resume, "job": job})

answer

"The resume highlights Luis' extensive experience in data science and machine learning, particularly in areas such as natural language processing, recommendation systems, marketing and customer relationship management, and time-series forecasting. He also has a strong background in building and maintaining databases, working with various technologies like Apache Spark, Hadoop, and Amazon Web Services. The job vacancy specifically mentions that the candidate should design, develop, and deploy machine learning models and algorithms for complex and unique datasets, using various techniques such as mathematical modeling, scikit-learn, NLP, CNN, RNN, DL, RL, Transformers, GAN, and LLM. The candidate must collaborate with cross-functional teams to extract insights, identify business opportunities, and provide data-driven recommendations. They should stay up-to-date with the latest machine learning and AI techniques and tools, participate in Kaggle competitions, and have excellent programming

# TESTS

## LLMware

In [ ]:
new_prompt = """\<human>\: JOB: Design, develop, and deploy machine learning models and algorithms for complex and unique datasets, using various techniques such as mathematical modeling, scikit-learn, NLP, CNN, RNN, DL, RL, Transformers, GAN, LLM, RAG
Collaborate with cross-functional teams to extract insights, identify business opportunities and provide data-driven recommendations
Stay up-to-date with the latest machine learning and AI techniques and tools
Communicate complex technical concepts to non-technical stakeholders in an easy-to-understand manner
Bachelor's degree or higher in Computer Science, Mathematics, Statistics, Actuarial Science, Informatics, Information Science or related fields
Strong analytical skills and attention to detail
Participation in Kaggle, Mathematics Olympiad or similar competitions is a plus
Excellent programming skills in Python, R, Java, or C++
Familiar with ML frameworks such as Tensorflow, Keras, PyTorch, MLFlow, AutoML, TensorRT, CUDA
Excellent communication and collaboration skills
Experience with designing, training, and deploying machine learning models
Customer centric and committed to deliver the best AI results to customers
 
RESUME: Luis Antonio Rodrigues is an accomplished data scientist and machine learning engineer with over eight years of experience in developing innovative machine learning products and services. He holds a BSc in Mathematics, an MSc, and a PhD in Mechanical Engineering from the University of Campinas, one of the most renowned universities in Latin America. Luis's expertise spans across various domains including Natural Language Processing (NLP), Recommender Systems, Marketing and CRM, and Time-Series Forecasting, with significant contributions across Banking, Consumer Packaged Goods, Retail, and Telecommunications industries.
Currently serving as a Principal Data Scientist at DEUS, an AI firm dedicated to human-centered solutions, Luis plays a crucial role in the development of a cutting-edge Retrieval-Augmented Generation (RAG) solution. His responsibilities include improving the knowledge-to-text module, optimizing information retrieval for efficiency and precision, and enhancing text generation for real-time accuracy,  showcasing his skills in RAG, IR, LLM, NLP, and several tools and platforms. Additionally, he has contributed as a Data Architect in designing a medallion architecture for a Databricks lakehouse on AWS.
Previously, Luis held the position of Principal Data Consultant at Aubay Portugal, where he led an NLP project for Banco de Portugal, focusing on AI services such as summarization, information extraction, complaint text classification, and financial sentiment analysis. At CI&T, as Lead Data Scientist, he was instrumental in developing a recommender system for Nestlé, resulting in a 6% sales increase. During his time at Propz, he developed a recommender system for Carrefour, which boosted revenue by 3%.
His earlier roles include a researcher at I.Systems, focusing on water distribution systems, and at the University of Campinas, where his work centered on system and control theory. Luis's proficiency is further demonstrated by his certifications in MLOps with Azure Machine Learning, TensorFlow 2.0, and Python for Time Series Data Analysis. Luis combines his deep technical knowledge with strong communication skills to lead teams and projects towards achieving significant business impacts.
Why is RESUME suitable for the JOB?
\<bot>\:"""


model_id = "llmware/dragon-yi-6b-v0"  
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)

inputs = tokenizer(new_prompt, return_tensors="pt")  
start_of_output = len(inputs.input_ids[0])

#   temperature: set at 0.3 for consistency of output
#   max_new_tokens:  set at 100 - may prematurely stop a few of the summaries

device = "cuda:0" if torch.cuda.is_available() else "cpu"

outputs = model.generate(
    inputs.input_ids.to(device),
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.3,
    max_new_tokens=100,
)

output = tokenizer.decode(outputs[0][start_of_output:],skip_special_tokens=True)  

#   note: due to artifact of the fine-tuning, use this post-processing with HF generation 
if "llmware" in model_id:
    eot = output.find("<|endoftext|>")
    if eot > -1:
        output = output[:eot]

output

## Mamba

In [ ]:
#!pip install torch==2.1.0 transformers==4.35.0 causal-conv1d==1.0.0 mamba-ssm==1.0.1

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

CHAT_TEMPLATE_ID = "HuggingFaceH4/zephyr-7b-beta"

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_name = "clibrain/mamba-2.8b-instruct-openhermes"

eos_token = "<|endoftext|>"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.eos_token = eos_token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained(CHAT_TEMPLATE_ID).chat_template

model = MambaLMHeadModel.from_pretrained(
        model_name, device=device, dtype=torch.float16)

messages = []
prompt = "Tell me 5 sites to visit in Spain"
messages.append(dict(role="user", content=prompt))

input_ids = tokenizer.apply_chat_template(
            messages, return_tensors="pt", add_generation_prompt=True
).to(device)

out = model.generate(
    input_ids=input_ids,
    max_length=2000,
    temperature=0.9,
    top_p=0.7,
    eos_token_id=tokenizer.eos_token_id,
)

decoded = tokenizer.batch_decode(out)
assistant_message = (
    decoded[0].split("<|assistant|>\n")[-1].replace(eos_token, "")
)

print(assistant_message)